In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math

In [4]:
train = pd.read_csv('data/trlast(EN)0421.csv')
test = pd.read_csv('data/telast(EN)0421.csv')

In [3]:
train

,date,company,title,text,nextclose,lastclose,groundtruth01
0,2017-12-27,tesla,Tesla to make pickup truck after Model Y cross...,Wed Dec 27 2017 / 4:49 PM EST Tesla to make p...,311.64,317.29,0.0
1,2018-01-01,google,Former Google career coach shares ways to ace ...,Getty Images SHARES Starting a new job can fe...,1073.21,1053.40,1.0
2,2018-01-01,amazon,The Limits of Amazon,Amazon.com Inc. is a colossus. In the near fut...,1189.01,1169.47,1.0
3,2018-01-01,amazon,Toyota Working With Amazon,Uber to Develop Self-Driving Shuttle | Fortune...,1189.01,1169.47,1.0
4,2018-01-02,amazon,BRIEF-Amazon.Com Says ‍More Than Five Bln Item...,34 PM / Updated 8 minutes ago BRIEF-Amazon.Com...,1189.01,1169.47,1.0
5,2018-01-02,facebook,Why Shares of Facebook,Amazon Netflix and Google Can Rise Further i...,181.42,176.46,1.0
6,2018-01-02,amazon,Why Shares of Facebook,Amazon Netflix and Google Can Rise Further i...,1189.01,1169.47,1.0
7,2018-01-02,google,Why Shares of Facebook,Amazon Netflix and Google Can Rise Further i...,1073.21,1053.40,1.0
8,2018-01-02,apple,CLSA: Apple iPhone X expectations 'still too h...,Apple iPhone X volume expectations are still t...,172.26,169.23,1.0
9,2018-01-02,apple,Sell Apple on battery life canard? No!: Jim Cr...,Sell Apple on battery life canard? No!: Jim Cr...,172.26,169.23,1.0


In [5]:
y_train = train['groundtruth01']
X_train = train['text']

In [8]:
positive = 0
negative = 0
total = 0
words_dict = {}
words_dict['positive_words'] = {}
words_dict['negative_words'] = {}
for index, news in enumerate(X_train): 
    if y_train[index] == 1:
        positive += 1
        cl = 'positive'
    elif y_train[index] == 0:
        negative += 1
        cl = 'negative'
    word_list = str(news).split(' ')
    word_list = [x.lower() for x in word_list]
    for word in word_list:
        if word != "" and word != " ":
            total += 1
            if word in words_dict[cl+'_words'].keys():
                words_dict[cl+'_words'][word] += 1
            else:
                words_dict[cl+'_words'][word] = 1

                    
positive_total = sum(words_dict['positive_words'].values())
negative_total = sum(words_dict['negative_words'].values())

prior_positive = positive/(positive+negative)
prior_negative = negative/(positive+negative)

all_words = set(list(words_dict['positive_words'].keys())+list(words_dict['negative_words'].keys()))
words_len = len(all_words)
for word in all_words:
    if word in words_dict['positive_words']:
        # Laplace smoothing
        positive_r = (words_dict['positive_words'][word] + 1)/(positive_total+words_len)
    else:
        positive_r = 1/(positive_total+words_len)
    if word in words_dict['negative_words']:
        negative_r = (words_dict['negative_words'][word] + 1)/(negative_total+words_len)
    else:
        negative_r = 1/(negative_total+words_len)
    words_dict['positive_words'][word] = positive_r
    words_dict['negative_words'][word] = negative_r

In [9]:
words_dict

{'positive_words': {'january': 0.0003609967174342479,
  '3': 0.0007989572020372186,
  '2018': 0.0012564640402068492,
  '/': 0.0008435473211111614,
  '11:03': 3.0541177447905914e-06,
  'am': 0.00042024660168318536,
  'updated': 0.0004373496610540127,
  'an': 0.0033491455189373627,
  'hour': 0.00016858729951244064,
  'ago': 0.0009687661486475756,
  'china': 0.000511259310477945,
  'to': 0.026331381548486563,
  'boost': 0.00012644047463433047,
  'intellectual': 7.635294361976478e-05,
  'property': 0.00011727812139995871,
  'protection:': 1.2216470979162365e-06,
  'state': 0.000258378361209284,
  'media': 0.0009162353234371774,
  'reuters': 0.000645029667699773,
  'staff': 0.0002699840086394883,
  '1': 0.0007244367290643283,
  'min': 0.0003909270713331957,
  'read': 0.0005662334298841756,
  'beijing': 4.214682487811016e-05,
  '(reuters)': 0.0009046296760069732,
  '-': 0.0019069911198472452,
  'china’s': 7.268800232601607e-05,
  'council': 3.481694229061274e-05,
  'said': 0.0050148613369461

In [10]:
prior_positive

0.5615084525357608

In [11]:
for i in range(len(test)):
    news = str(test.loc[i,'text'])
    positive_p = math.log10(prior_positive)
    negative_p = math.log10(prior_negative)
    word_list = news.split(' ')
    word_list = [x.lower() for x in word_list]
    for word in word_list:
        if word != "" and word != " " and word in words_dict['positive_words']:
            positive_p = positive_p+math.log10(words_dict['positive_words'][word])
            negative_p = negative_p+math.log10(words_dict['negative_words'][word])
    if positive_p == max(positive_p, negative_p):
        label = 1
    elif negative_p == max(positive_p, negative_p):
        label = 0
    test.loc[i,'nbpred'] = label

In [65]:
preds = []
for index, news in enumerate(X_test):
    groundtruth = y_test[index]
    positive_p = math.log10(prior_positive)
    negative_p = math.log10(prior_negative)
    word_list = list(jieba.cut(news))
    for word in word_list:
        if word != "" and word != " " and word in words_dict['positive_words']:
            positive_p = positive_p+math.log10(words_dict['positive_words'][word])
            negative_p = negative_p+math.log10(words_dict['negative_words'][word])
    if positive_p == max(positive_p, negative_p):
        label = 1
    elif negative_p == max(positive_p, negative_p):
        label = 0
    preds.append(label)

In [12]:
from sklearn.metrics import f1_score
f1_score(test['nbpred'], test['groundtruth01'])

0.766637856525497

In [18]:
test.to_csv('data/NBtest(EN).csv',index=False)

In [16]:
test

,date,company,title,text,nextclose,lastclose,timeseriesforecast,timeserieslower,timeseriesupper,groundtruth01,timeseries01,nbpred
0,2017-12-27,apple,Apple Watch well being options reveal new fron...,Author Natalia Camp Published December 27 201...,311.64,317.29,321.076679,313.307849,328.845508,0.0,1.0,1.0
1,2018-01-01,google,A Browser You’ve Never Heard of Is Dethroning ...,JAKARTA Indonesia—A mobile browser rarely use...,1073.21,1053.40,1053.122599,1039.585449,1066.659749,1.0,0.0,1.0
2,2018-01-02,apple,We see Apple doing more 'add-on' acquisitions:...,We see Apple doing more 'add-on' acquisitions:...,1189.01,1169.47,1173.742521,1153.901074,1193.583968,1.0,1.0,0.0
3,2018-01-02,intel,Inspirata Acquires Toronto-based Artificial In...,Inc. (AIM) to Enhance its R&D in Artificial In...,1189.01,1169.47,1173.742521,1153.901074,1193.583968,1.0,1.0,1.0
4,2018-01-02,google,The Wall Street Journal: The Alibaba browser n...,Published: Jan 2 2018 8:32 a.m. ET Share UC ...,1189.01,1169.47,1173.742521,1153.901074,1193.583968,1.0,1.0,1.0
5,2018-01-02,amazon,Amazon will buy Target in 2018,influential tech analyst Gene Munster predicts...,181.42,176.46,176.314252,172.420451,180.208052,1.0,0.0,1.0
6,2018-01-03,google,Amazon and Google cut speaker prices in market...,analysts say Amazon.com and Alphabet's Google ...,1204.20,1189.01,1189.709884,1169.323951,1210.095818,1.0,1.0,1.0
7,2018-01-03,facebook,Facebook to Announce Fourth Quarter and Full Y...,MENLO PARK Calif. Jan. 2 2018 /PRNewswire/ ...,1091.52,1073.21,1087.705184,1072.445583,1102.964784,1.0,1.0,1.0
8,2018-01-03,amazon,Amazon grabbed 4 percent of all US retail sale...,There's no denying that Amazon 's retail busin...,172.23,172.26,173.185404,170.231365,176.139443,0.0,1.0,1.0
9,2018-01-03,google,Google discovers 'serious' flaws in Intel and ...,SANTA CLARA Calif. (AP) — Security researcher...,172.23,172.26,173.185404,170.231365,176.139443,0.0,1.0,0.0
